In [3]:
from typing import List

from langchain.output_parsers import StructuredOutputParser, ResponseSchema, PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from pydantic import BaseModel, Field

In [9]:
# Here's another example, but with a compound typed field.
class WeightEstimation(BaseModel):
    ingredient: str = Field(description="The name of the ingredient")
    calculation: str = Field(description="The calculation used to estimate the weight")
    weight: float = Field(description="The estimated weight of the ingredient in kg.")
    negligible: bool = Field(description="Whether the ingredient is negligible in weight. Is true if the weight is less than 0.1 kg.")


actor_query = "Generate the filmography for a random actor."

parser = PydanticOutputParser(pydantic_object=WeightEstimation)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

_input = prompt.format_prompt(query=actor_query)

print(_input.text)

Answer the user query.
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredient": {"title": "Ingredient", "description": "The name of the ingredient", "type": "string"}, "calculation": {"title": "Calculation", "description": "The calculation used to estimate the weight", "type": "string"}, "weight": {"title": "Weight", "description": "The estimated weight of the ingredient in kg.", "type": "number"}, "negligible": {"title": "Negligible", "description": "Whether the ingredient is negligible in weight. Is true if the weight is less than 0.1 kg.", "type": "boolean"}}, "requ

In [8]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "name of an actor", "type": "string"}, "film_names": {"title": "Film Names", "description": "list of names of films they starred in", "type": "array", "items": {"type": "string"}}}, "required": ["name", "film_names"]}\n```'